In [ ]:
import pymssql
import pandas.io.sql as sqlio
import warnings

warnings.filterwarnings('ignore')

conn = pymssql.connect(
    host='macq-poc.ctrfy31kg8iq.ap-southeast-2.rds.amazonaws.com',
    database='master',
    user='admin',
    port=1433,
    password='',
)

In [ ]:
def get_query(par_conn):
    sql = '''
        SELECT ItemID, Path
        FROM rdsadmin_ReportServer.dbo.Catalog
        where Name = 'Sales Orders'
        '''
    data = sqlio.read_sql_query(sql, par_conn)
    return data

query = get_query(conn)
print(query)

In [ ]:

def get_query(par_conn):
    sql = '''
        select convert(varchar(max), convert(varbinary(max), content))
        from rdsadmin_ReportServer.dbo.Catalog
        where Name = 'Sales Orders'
        '''
    data = sqlio.read_sql_query(sql, par_conn)
    return data

query = get_query(conn)
print(query)

In [ ]:

def get_query(par_conn):
    sql = '''
        SELECT
        ItemID -- Unique Identifier
        , [Path] --Path including object name
        , [Name] --Just the objectd name
        , ParentID --The ItemID of the folder in which it resides
        , CASE [Type] --Type, an int which can be converted using this case statement.
            WHEN 1 THEN 'Folder'
            WHEN 2 THEN 'Report'
            WHEN 3 THEN 'File'
            WHEN 4 THEN 'Linked Report'
            WHEN 5 THEN 'Data Source'
            WHEN 6 THEN 'Report Model - Rare'
            WHEN 7 THEN 'Report Part - Rare'
            WHEN 8 THEN 'Shared Data Set - Rare'
            WHEN 9 THEN 'Image'
            ELSE CAST(Type as varchar(100))
        END AS TypeName
        --, content
        , LinkSourceID --If a linked report then this is the ItemID of the actual report.
        , [Description] --This is the same information as can be found in the GUI
        , [Hidden] --Is the object hidden on the screen or not
        , CreatedBy.UserName CreatedBy
        , CreationDate
        , ModifiedBy.UserName ModifiedBy
        , CTG.ModifiedDate
        FROM 
       rdsadmin_ReportServer.dbo.[Catalog] CTG
            INNER JOIN 
       rdsadmin_ReportServer.dbo.Users CreatedBy ON CTG.CreatedByID = CreatedBy.UserID
            INNER JOIN
       rdsadmin_ReportServer.dbo.Users ModifiedBy ON CTG.ModifiedByID = ModifiedBy.UserID
        '''
    data = sqlio.read_sql_query(sql, par_conn)
    return data

query = get_query(conn)
print(query)

In [ ]:
def get_query(par_conn):
    sql = '''
       SELECT
        [Path]
      , CASE [Type]
          WHEN 2 THEN 'Report'
          WHEN 5 THEN 'Data Source'    
        END AS TypeName
      , CAST(CAST(content AS varbinary(max)) AS xml) [Query Report]
      , [Description]
      FROM rdsadmin_ReportServer.dbo.[Catalog] CTG
      WHERE
        [Type] IN (2, 5);
        '''
    data = sqlio.read_sql_query(sql, par_conn)
    return data

query = get_query(conn)
print(query)

In [ ]:
def get_query(par_conn):
    sql = '''

        '''
    data = sqlio.read_sql_query(sql, par_conn)
    return data

query = get_query(conn)


print(query)

In [4]:
%reload_ext dotenv
%dotenv

import os
import requests
from requests_ntlm import HttpNtlmAuth

# SSRS server details and Report variables
report_server_url = os.environ.get("REPORT_SERVER_URL")
report_path = os.environ.get("REPORT_PATH")
cert_path = os.environ.get("CERT_PATH")
report_name = os.environ.get("REPORT_NAME")
export_format = os.environ.get("EXPORT_FORMAT")

username = os.environ.get("USERNAME")
password = os.environ.get("PASSWORD")
domain = os.environ.get("DOMAIN")

order_id = os.environ.get("ORDER_ID")
date_start = os.environ.get("DATE_START")
date_end = os.environ.get("DATE_END")

# Create a session with NTLM authentication
session = requests.Session()
session.auth = HttpNtlmAuth(f'{domain}\\{username}', password)
session.verify = cert_path

params = {'OrderID':f'{order_id}', 
          'DateStart':f'{date_start}', 
          'DateEnd':f'{date_end}'}

export_format = f'{export_format}'
request_url = (f'{report_server_url}'
               f'%2f{report_path}'
               f'&rs:Format={export_format}'
               f'&rs:ParameterLanguage=en-US'
               f'&rs:Command=Render')

response = session.get(request_url, params=params, stream=True)
with open(f'{report_name}.{export_format}', 'wb') as f:
    f.write(response.content)
    print(report_path)
    print(order_id)
    print(request_url)
    print(response)

SalesOrdersPar
SO43664
https://macq-poc.ctrfy31kg8iq.ap-southeast-2.rds.amazonaws.com:8443/ReportServer/Pages/ReportViewer.aspx?%2fSalesOrdersPar&rs:Format=csv&rs:ParameterLanguage=en-US&rs:Command=Render
<Response [200]>


In [3]:
%reload_ext dotenv
%dotenv

import os
import requests
from requests_ntlm import HttpNtlmAuth

# SSRS server details
report_server_url = os.environ.get("REPORT_SERVER_URL")
report_path = os.environ.get("REPORT_PATH")
cert_path = os.environ.get("CERT_PATH")
report_name = os.environ.get("REPORT_NAME")
export_format = os.environ.get("EXPORT_FORMAT")

username = os.environ.get("USERNAME")
password = os.environ.get("PASSWORD")
domain = os.environ.get("DOMAIN")

order_id = os.environ.get("ORDER_ID")
date_start = os.environ.get("DATE_START")
date_end = os.environ.get("DATE_END")

# Create a session with NTLM authentication
session = requests.Session()
session.auth = HttpNtlmAuth(f'{domain}\\{username}', password)
session.verify = cert_path

export_format = f'{export_format}'
request_url = (f'{report_server_url}'
               f'%2f{report_path}'
               f'&rs:Format={export_format}'
               f'&rs:ParameterLanguage=en-US'
               f'&rs:Command=Render')

params = {'OrderID':'SO43664', 'date_start':'1/1/2011', 'date_end':'1/1/2015'}

import xml.etree.ElementTree as ET

# Build the XML message
soap_request = ET.Element('soapenv:Envelope', {'xmlns:soapenv': 'http://schemas.xmlsoap.org/soap/envelope/',
                                               'xmlns:ser': 'http://schemas.microsoft.com/sqlserver/2005/06/30/reporting/reportingservices'})
soap_header = ET.SubElement(soap_request, 'soapenv:Header')
soap_body = ET.SubElement(soap_request, 'soapenv:Body')
exec_request = ET.SubElement(soap_body, 'ser:ExecuteReport')
exec_request.set('xmlns', 'http://schemas.microsoft.com/sqlserver/2005/06/30/reporting/reportingservices')

# Set the report path
report_path_element = ET.SubElement(exec_request, 'ser:Report')
report_path_element.text = report_path

# Set the parameter values
params_element = ET.SubElement(exec_request, 'ser:Parameters')
for param_name, param_value in params.items():
    param_element = ET.SubElement(params_element, 'ser:Parameter')
    param_element_name = ET.SubElement(param_element, 'ser:Name')
    param_element_name.text = param_name
    param_element_value = ET.SubElement(param_element, 'ser:Value')
    param_element_value.text = param_value

# Set the other required parameters
render_format = ET.SubElement(exec_request, 'ser:RenderFormat')
render_format.text = 'PDF'
history_id = ET.SubElement(exec_request, 'ser:HistoryID')
history_id.text = ''
# END OF SOAP

headers = {'Content-Type': 'text/xml;charset=UTF-8', 'SOAPAction': 'http://schemas.microsoft.com/sqlserver/2005/06/30/reporting/reportingservices/ExecuteReport'}
response = requests.post(request_url, data=ET.tostring(soap_request), headers=headers, auth=session.auth, verify = session.verify)

if response.status_code == 200:
    with open('report.pdf', 'wb') as f:
        f.write(response.content)
        print(request_url)
        print(response.content)
        print(response)
else:
    print('Error: Failed to generate report')



# response = session.get(request_url, stream=True)
# with open(f'{report_name}.{export_format}', 'wb') as f:
#     # f.write(response.content)
#     print(report_path)
#     print(request_url)
#     print(response)

https://macq-poc.ctrfy31kg8iq.ap-southeast-2.rds.amazonaws.com:8443/ReportServer/Pages/ReportViewer.aspx?%2fSales+Orders&rs:Format=csv&rs:ParameterLanguage=en-US&rs:Command=Render
b''
<Response [200]>


In [11]:
response = session.get(request_url, stream=True)
with open(f'{report_name}.{export_format}', 'wb') as f:
    # f.write(response.content)
    print(report_path)
    print(request_url)
    print(response)